<a href="https://colab.research.google.com/github/snehithmekala/express-exercise/blob/main/VAE_transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/fialle/Anomaly-VAE-Transformer.git


fatal: destination path 'Anomaly-VAE-Transformer' already exists and is not an empty directory.


In [2]:
%cd Anomaly-VAE-Transformer


/content/Anomaly-VAE-Transformer


In [3]:
!pip install -r requirements.txt


ERROR: Invalid requirement: 'absl-py=0.15.0=pypi_0': Expected end or semicolon (after name and no valid version specifier)
    absl-py=0.15.0=pypi_0
           ^ (from line 4 of requirements.txt)
Hint: = is not a valid operator. Did you mean == ?


In [4]:
%cd /content/Anomaly-VAE-Transformer


/content/Anomaly-VAE-Transformer


In [5]:
!sed 's/=/==/2; s/==pypi_0//' requirements.txt > fixed_requirements.txt


In [6]:
!pip install -r fixed_requirements.txt


ERROR: Invalid requirement: 'absl-py=0.15.0': Expected end or semicolon (after name and no valid version specifier)
    absl-py=0.15.0
           ^ (from line 4 of fixed_requirements.txt)
Hint: = is not a valid operator. Did you mean == ?


In [7]:
!sed -E 's/([a-zA-Z0-9_-]+)=([0-9])/\1==\2/g; s/==pypi_0//g; s/==conda_forge//g' requirements.txt > pip_requirements.txt


In [8]:
!head pip_requirements.txt


# This file may be used to create an environment using:
# $ conda create --name <env> --file <this file>
# platform: win-64
absl-py==0.15.0=pypi_0
astunparse==1.6.3=pypi_0
backcall==0.2.0=pyh9f0ad1d_0
backports==1.0=pyhd8ed1ab_3
backports.functools_lru_cache==1.6.4=pyhd8ed1ab_0
blas==1.0=mkl
bottleneck==1.3.5=py37h080aedc_0


In [9]:
!pip install -r pip_requirements.txt


ERROR: Invalid requirement: 'absl-py==0.15.0=pypi_0': Expected end or semicolon (after version specifier)
    absl-py==0.15.0=pypi_0
           ~~~~~~~~^ (from line 4 of pip_requirements.txt)


In [10]:
!pip install torch torchvision torchaudio
!pip install numpy pandas matplotlib seaborn scikit-learn tqdm absl-py einops
!pip install tensorflow tensorboard


In [83]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, LSTM, RepeatVector, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K

class TimeVAE:
    def __init__(self, timesteps, features, latent_dim=16, hidden_dim=64):
        self.timesteps = timesteps
        self.features = features
        self.latent_dim = latent_dim
        self.hidden_dim = hidden_dim

        self.encoder = self._get_encoder()
        self.decoder = self._get_decoder()

        inputs = Input(shape=(timesteps, features))
        z_mean, z_log_var, z = self.encoder(inputs)
        reconstructed = self.decoder(z)
        self.model = Model(inputs, reconstructed)

        # VAE Loss
        reconstruction_loss = tf.reduce_mean(tf.square(inputs - reconstructed))
        kl_loss = -0.5 * tf.reduce_mean(
            1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var)
        )
        vae_loss = reconstruction_loss + kl_loss
        self.model.add_loss(vae_loss)
        self.model.compile(optimizer='adam')

    # --------------------------
    # Encoder
    # --------------------------
    def _get_encoder(self):
        inputs = Input(shape=(self.timesteps, self.features))
        x = LSTM(self.hidden_dim, activation='relu')(inputs)
        x = Dense(self.hidden_dim, activation='relu')(x)

        # fix: use shape instead of get_shape()
        self.encoder_last_dense_dim = x.shape[-1]

        z_mean = Dense(self.latent_dim, name='z_mean')(x)
        z_log_var = Dense(self.latent_dim, name='z_log_var')(x)

        def sampling(args):
            z_mean, z_log_var = args
            epsilon = K.random_normal(shape=(K.shape(z_mean)[0], self.latent_dim))
            return z_mean + K.exp(0.5 * z_log_var) * epsilon

        z = tf.keras.layers.Lambda(sampling, output_shape=(self.latent_dim,), name='z')([z_mean, z_log_var])
        return Model(inputs, [z_mean, z_log_var, z], name='encoder')

    # --------------------------
    # Decoder
    # --------------------------
    def _get_decoder(self):
        latent_inputs = Input(shape=(self.latent_dim,))
        x = RepeatVector(self.timesteps)(latent_inputs)
        x = LSTM(self.hidden_dim, activation='relu', return_sequences=True)(x)
        outputs = TimeDistributed(Dense(self.features))(x)
        return Model(latent_inputs, outputs, name='decoder')


In [86]:
from TimeVAE import TimeVAE

# ✅ include the required hidden_layer_sizes argument
vae = TimeVAE(
    timesteps=1,
    features=64,
    latent_dim=8,
    hidden_layer_sizes=[128, 64]   # 👈 add this list (required argument)
)

vae.model.summary()






TypeError: BaseVariationalAutoencoder.__init__() missing 2 required positional arguments: 'seq_len' and 'feat_dim'

In [11]:
!ls


code  fixed_requirements.txt  pip_requirements.txt  requirements.txt
fig   LICENSE		      README.md


In [12]:
!ls code


 analysis.ipynb		    solver.py
 anomaly_score.npy	    Testing_filled_MITM_numeric.csv
'anomaly score_paper.png'   tf_loss.png
 anomaly_score.png	    tf_output.npy
 case1.png		    tf_train_loss.npy
 case2.png		    tf_val_loss.npy
 case3.png		    TimeVAE.py
 case4.png		    trainer_VAE.py
 checkpoints		    transformer_loss.npy
 data_loader.py		    transformer_loss.png
 datasets		    Transformer.py
 embed.py		    utils.py
 encoded_data.npy	    vae_base.py
 histogram.png		    vae_loss.npy
 loss_test_data.npy	    vae_loss.png
 loss_train_data.npy	    vae_model
 main.py		    VaeTransformer.yaml
 process.ipynb		    visualize.ipynb
 __pycache__


In [13]:
!python code/train.py --dataset ECG


python3: can't open file '/content/Anomaly-VAE-Transformer/code/train.py': [Errno 2] No such file or directory


In [14]:
!ls -R


.:
code  fixed_requirements.txt  pip_requirements.txt  requirements.txt
fig   LICENSE		      README.md

./code:
 analysis.ipynb		    solver.py
 anomaly_score.npy	    Testing_filled_MITM_numeric.csv
'anomaly score_paper.png'   tf_loss.png
 anomaly_score.png	    tf_output.npy
 case1.png		    tf_train_loss.npy
 case2.png		    tf_val_loss.npy
 case3.png		    TimeVAE.py
 case4.png		    trainer_VAE.py
 checkpoints		    transformer_loss.npy
 data_loader.py		    transformer_loss.png
 datasets		    Transformer.py
 embed.py		    utils.py
 encoded_data.npy	    vae_base.py
 histogram.png		    vae_loss.npy
 loss_test_data.npy	    vae_loss.png
 loss_train_data.npy	    vae_model
 main.py		    VaeTransformer.yaml
 process.ipynb		    visualize.ipynb
 __pycache__

./code/checkpoints:
Olympus_VAE_checkpoint.pth

./code/datasets:
daily_test_label.npy  encoded_test.npy	     hourly_test.npy
daily_test.npy	      encoded_train.npy      hourly_total.csv
daily_train.npy       hourly_test_label.npy  hourly_train

In [15]:
!python main.py


python3: can't open file '/content/Anomaly-VAE-Transformer/main.py': [Errno 2] No such file or directory


In [16]:
%cd /content/Anomaly-VAE-Transformer/code


/content/Anomaly-VAE-Transformer/code


In [50]:
from trainer_VAE import VAETrainer
from solver import Solver
import torch
import pandas as pd



In [18]:
!sed -i 's/np\.Inf/np.inf/g' /content/Anomaly-VAE-Transformer/code/solver.py


In [68]:
from google.colab import files
uploaded = files.upload()


Saving ton-iot.csv to ton-iot (1).csv


In [69]:
import pandas as pd

df = pd.read_csv("ton-iot.csv")
print(df.head())


           ts         src_ip  src_port         dst_ip  dst_port proto service  \
0  1554198358    3.122.49.24      1883  192.168.1.152     52976   tcp       -   
1  1554198358   192.168.1.79     47260  192.168.1.255     15600   udp       -   
2  1554198359  192.168.1.152      1880  192.168.1.152     51782   tcp       -   
3  1554198359  192.168.1.152     34296  192.168.1.152     10502   tcp       -   
4  1554198362  192.168.1.152     46608  192.168.1.190        53   udp     dns   

       duration  src_bytes  dst_bytes  ... http_response_body_len  \
0  80549.530260    1762852   41933215  ...                      0   
1      0.000000          0          0  ...                      0   
2      0.000000          0          0  ...                      0   
3      0.000000          0          0  ...                      0   
4      0.000549          0        298  ...                      0   

   http_status_code  http_user_agent  http_orig_mime_types  \
0                 0                -

In [21]:
!ls /content/Anomaly-VAE-Transformer/code/checkpoints



Olympus_VAE_checkpoint.pth


In [22]:
!ls


 analysis.ipynb		    solver.py
 anomaly_score.npy	   'Testing_filled_MITM_numeric (1).csv'
'anomaly score_paper.png'   Testing_filled_MITM_numeric.csv
 anomaly_score.png	    tf_loss.png
 case1.png		    tf_output.npy
 case2.png		    tf_train_loss.npy
 case3.png		    tf_val_loss.npy
 case4.png		    TimeVAE.py
 checkpoints		    trainer_VAE.py
 data_loader.py		    transformer_loss.npy
 datasets		    transformer_loss.png
 embed.py		    Transformer.py
 encoded_data.npy	    utils.py
 histogram.png		    vae_base.py
 loss_test_data.npy	    vae_loss.npy
 loss_train_data.npy	    vae_loss.png
 main.py		    vae_model
 process.ipynb		    VaeTransformer.yaml
 __pycache__		    visualize.ipynb


In [23]:
import torch
from trainer_VAE import VAETrainer
from solver import Solver   # ✅ no 'code.' here
import pandas as pd



In [44]:
!cat solver.py

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import os
import time

from Transformer import TransformerModel
from data_loader import get_loader_segment
import matplotlib.pyplot as plt

def adjust_learning_rate(optimizer, epoch, lr_):
    lr_adjust = {epoch: lr_ * (0.5 ** ((epoch - 1) // 1))}
    if epoch in lr_adjust.keys():
        lr = lr_adjust[epoch]
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr
        print('Updating learning rate to {}'.format(lr))


class EarlyStopping:
    def __init__(self, patience=7, verbose=False, dataset_name='', delta=0):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.best_score2 = None
        self.early_stop = False
        self.val_loss_min = np.inf
        self.val_loss2_min = np.inf
        self.delta = delta
        self.dataset = dataset_name

    def __call__(self, val_loss, val_los

In [34]:
!head -n 40 /content/Anomaly-VAE-Transformer/code/trainer_VAE.py


import os, warnings
warnings.filterwarnings('ignore') 
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from TimeVAE import TimeVAE
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import joblib
import tensorflow as tf
#import utils


class VAETrainer():
    DEFAULTS = {}

    def __init__(self, config):

        self.__dict__.update(VAETrainer.DEFAULTS, **config)

        self.data_dir = './datasets/'
        self.train_file = 'hourly_train.npy'
        self.test_file = 'hourly_test.npy'

        self.win_size = 24
        self.n_feature = 12
        self.batch_size = 32

        self.train, self.val, self.test = self.get_data(self.data_dir, self.train_file, self.test_file)

        early_stop_loss = 'loss'
        self.early_stop_callback = EarlyStopping(monitor=ear

In [74]:
import pandas as pd

# Load your input data
df = pd.read_csv("ton-iot.csv")

print("Data loaded successfully!")
print(df.shape)
print(df.head())


Data loaded successfully!
(19, 45)
           ts         src_ip  src_port         dst_ip  dst_port proto service  \
0  1554198358    3.122.49.24      1883  192.168.1.152     52976   tcp       -   
1  1554198358   192.168.1.79     47260  192.168.1.255     15600   udp       -   
2  1554198359  192.168.1.152      1880  192.168.1.152     51782   tcp       -   
3  1554198359  192.168.1.152     34296  192.168.1.152     10502   tcp       -   
4  1554198362  192.168.1.152     46608  192.168.1.190        53   udp     dns   

       duration  src_bytes  dst_bytes  ... http_response_body_len  \
0  80549.530260    1762852   41933215  ...                      0   
1      0.000000          0          0  ...                      0   
2      0.000000          0          0  ...                      0   
3      0.000000          0          0  ...                      0   
4      0.000549          0        298  ...                      0   

   http_status_code  http_user_agent  http_orig_mime_types  \
0

In [71]:
import torch

# Load the saved model checkpoint
checkpoint_path = "checkpoints/Olympus_VAE_checkpoint.pth"
checkpoint = torch.load(checkpoint_path, map_location='cpu')

print("Checkpoint loaded successfully!")
print(checkpoint.keys())  # just to inspect what’s inside


Checkpoint loaded successfully!
odict_keys(['transformer_encoder.layers.0.self_attn.in_proj_weight', 'transformer_encoder.layers.0.self_attn.in_proj_bias', 'transformer_encoder.layers.0.self_attn.out_proj.weight', 'transformer_encoder.layers.0.self_attn.out_proj.bias', 'transformer_encoder.layers.0.linear1.weight', 'transformer_encoder.layers.0.linear1.bias', 'transformer_encoder.layers.0.linear2.weight', 'transformer_encoder.layers.0.linear2.bias', 'transformer_encoder.layers.0.norm1.weight', 'transformer_encoder.layers.0.norm1.bias', 'transformer_encoder.layers.0.norm2.weight', 'transformer_encoder.layers.0.norm2.bias', 'transformer_encoder.layers.1.self_attn.in_proj_weight', 'transformer_encoder.layers.1.self_attn.in_proj_bias', 'transformer_encoder.layers.1.self_attn.out_proj.weight', 'transformer_encoder.layers.1.self_attn.out_proj.bias', 'transformer_encoder.layers.1.linear1.weight', 'transformer_encoder.layers.1.linear1.bias', 'transformer_encoder.layers.1.linear2.weight', 'tran

In [75]:
import torch
from solver import Solver

# ✅ Configuration expected by Solver
config = {
    "batch_size": 32,
    "win_size": 24,
    "dataset": "Olympus",
    "data_path": "/content/Anomaly-VAE-Transformer/code/datasets",
    "train": True,
    "test": False,
    "input_c": 64,   # Number of features in your CSV (you showed 46 columns)
    "lr": 0.0001
}

# ✅ Initialize Solver safely
solver = Solver(config)

# ✅ Load checkpoint
checkpoint_path = "/content/Anomaly-VAE-Transformer/code/checkpoints/Olympus_VAE_checkpoint.pth"
model_state = torch.load(checkpoint_path, map_location=torch.device('cpu'))

# Load weights into correct model
if hasattr(solver, 'vae'):
    solver.vae.load_state_dict(model_state)
    print("✅ Weights loaded into solver.vae")
elif hasattr(solver, 'model'):
    solver.model.load_state_dict(model_state)
    print("✅ Weights loaded into solver.model")
else:
    print("⚠️ Model attribute not found; weights loaded as raw state_dict")

print("✅ Checkpoint loaded successfully!")


✅ Weights loaded into solver.model
✅ Checkpoint loaded successfully!


In [76]:
import numpy as np, pandas as pd, torch
from torch.utils.data import DataLoader, TensorDataset

# ✅ Load your CSV file
csv_name = "ton-iot.csv"   # change if needed
df = pd.read_csv(csv_name)
print("CSV loaded:", df.shape)

# ✅ Keep only numeric columns for model input
numeric_df = df.select_dtypes(include=[np.number]).copy()
n_features = numeric_df.shape[1]

# ✅ Pad to 64 features if fewer
if n_features < 64:
    for i in range(64 - n_features):
        numeric_df[f'pad_{i}'] = 0.0
elif n_features > 64:
    numeric_df = numeric_df.iloc[:, :64]
print("Numeric features shape after padding:", numeric_df.shape)

# ✅ Prepare windowed data (default win_size = 24)
win_size = 1
X = numeric_df.values.astype(np.float32)
rows = X.shape[0]
windows = []
for start in range(0, rows - win_size + 1):
    w = X[start:start+win_size, :]
    windows.append(w)
windows = np.stack(windows, axis=0)
print("Windows shape:", windows.shape)

# ✅ Convert to tensor
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tensor_data = torch.tensor(windows, dtype=torch.float32).to(device)
loader = DataLoader(TensorDataset(tensor_data), batch_size=32, shuffle=False)

# ✅ Run inference
solver.model.eval()
all_losses = []
with torch.no_grad():
    for (batch,) in loader:
        output = solver.model(batch, src_mask=None)
        mse = torch.mean((output - batch)**2, dim=(1,2))
        all_losses.append(mse.cpu().numpy())
all_losses = np.concatenate(all_losses)

# ✅ Map window-level losses to row-level scores
row_scores = np.zeros(rows)
counts = np.zeros(rows)
for i in range(len(all_losses)):
    for j in range(i, i + win_size):
        if j < rows:
            row_scores[j] += all_losses[i]
            counts[j] += 1
counts[counts == 0] = 1
row_scores = row_scores / counts

# ✅ Threshold to decide Attack / Normal
threshold = row_scores.mean() + row_scores.std()
preds = np.where(row_scores > threshold, "Attack", "Normal")

# ✅ If label column exists, compute accuracy
label_col = None
for name in ['label','attack','attack_type','Label']:
    if name in df.columns:
        label_col = name
        break

if label_col:
    y_true = df[label_col].astype(str).str.lower()
    y_true_bin = np.where(y_true.isin(['normal','benign','0']), 0, 1)
    pred_bin = np.where(preds == "Attack", 1, 0)
    acc = (y_true_bin == pred_bin).mean() * 100
    print(f"✅ Accuracy: {acc:.2f}%")
else:
    print("⚠️ No label column found — only showing predictions.")

# ✅ Save predictions
df['anomaly_score'] = row_scores
df['prediction'] = preds
df.to_csv("predictions_with_scores.csv", index=False)
print("✅ Saved: predictions_with_scores.csv")
print(df[['prediction','anomaly_score']].head(10))


CSV loaded: (19, 45)
Numeric features shape after padding: (19, 64)
Windows shape: (19, 1, 64)
✅ Accuracy: 42.11%
✅ Saved: predictions_with_scores.csv
  prediction  anomaly_score
0     Normal   3.777370e+16
1     Normal   3.774270e+16
2     Normal   3.774270e+16
3     Normal   3.774270e+16
4     Normal   3.774270e+16
5     Normal   3.774270e+16
6     Normal   3.774270e+16
7     Normal   3.774270e+16
8     Normal   3.784016e+16
9     Normal   3.784016e+16
